## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 1.30.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [1]:
!mkdir -p data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -O data/green_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet -O data/green_tripdata_2023-02.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet -O data/green_tripdata_2023-03.parquet

--2025-05-19 14:54:23--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.167.84.86, 3.167.84.131, 3.167.84.127, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.167.84.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2023-01.parquet’

data/green_tripdata 100%[===================>]   1.36M  --.-KB/s    in 0.02s   

2025-05-19 14:54:24 (86.6 MB/s) - ‘data/green_tripdata_2023-01.parquet’ saved [1427002/1427002]

--2025-05-19 14:54:24--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.167.84.127, 3.167.84.228, 3.167.84.131, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.167.84.127|:443... connected.
HTTP request sen

In [2]:
!python preprocess_data.py --raw_data_path data --dest_path ./output

In [3]:
!ls output

dv.pkl	test.pkl  train.pkl  val.pkl


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10

In [4]:
!mlflow ui

[2025-05-19 15:09:08 +0000] [22845] [INFO] Starting gunicorn 20.1.0
[2025-05-19 15:09:08 +0000] [22845] [INFO] Listening at: http://127.0.0.1:5000 (22845)
[2025-05-19 15:09:08 +0000] [22845] [INFO] Using worker: sync
[2025-05-19 15:09:08 +0000] [22852] [INFO] Booting worker with pid: 22852
[2025-05-19 15:09:47 +0000] [22845] [INFO] Handling signal: int
^C

Aborted!
[2025-05-19 15:09:47 +0000] [22852] [INFO] Worker exiting (pid: 22852)


In [3]:
!python train.py

2025/05/19 17:31:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/19 17:31:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [5]:
import mlflow
from mlflow.tracking import MlflowClient

# Connect to your tracking server
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Create client
client = MlflowClient()

# Get experiment by name
experiment = client.get_experiment_by_name("random-forest-training-2025")
experiment_id = experiment.experiment_id

# List runs in experiment
runs = client.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.rmse ASC"]  # Optional: sort by a metric
)

# Access run data
for run in runs:
    run_id = run.info.run_id
    metrics = run.data.metrics
    params = run.data.params
    
    print(f"min_samples_split: {params['min_samples_split']}")

min_samples_split: 2


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

In [7]:
!mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts

[2025-05-19 17:38:38 +0000] [14331] [INFO] Starting gunicorn 20.1.0
[2025-05-19 17:38:38 +0000] [14331] [INFO] Listening at: http://127.0.0.1:5000 (14331)
[2025-05-19 17:38:38 +0000] [14331] [INFO] Using worker: sync
[2025-05-19 17:38:38 +0000] [14333] [INFO] Booting worker with pid: 14333
[2025-05-19 17:38:38 +0000] [14334] [INFO] Booting worker with pid: 14334
[2025-05-19 17:38:38 +0000] [14335] [INFO] Booting worker with pid: 14335
[2025-05-19 17:38:38 +0000] [14336] [INFO] Booting worker with pid: 14336
[2025-05-19 17:38:47 +0000] [14331] [INFO] Handling signal: int
^C

Aborted!
[2025-05-19 17:38:47 +0000] [14334] [INFO] Worker exiting (pid: 14334)
[2025-05-19 17:38:47 +0000] [14335] [INFO] Worker exiting (pid: 14335)
[2025-05-19 17:38:47 +0000] [14336] [INFO] Worker exiting (pid: 14336)
[2025-05-19 17:38:47 +0000] [14333] [INFO] Worker exiting (pid: 14333)


## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818

In [8]:
!python hpo.py

2025/05/19 17:45:42 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|██████████| 15/15 [00:56<00:00,  3.74s/trial, best loss: 5.335419588556921]


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [11]:
!python register_model.py

2025/05/19 17:59:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Registered model 'rf-best-model' already exists. Creating a new version of this model...
2025/05/19 17:59:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: rf-best-model, version 4
Created version '4' of model 'rf-best-model'.


In [12]:
import mlflow
from mlflow.tracking import MlflowClient

# Connect to your tracking server
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Create client
client = MlflowClient()

# Get the latest version of the registered model
model_name = "rf-best-model"
latest_version = client.get_latest_versions(model_name, stages=["None"])[0]
run_id = latest_version.run_id

# Get the metrics from the run that created this model
run = client.get_run(run_id)
test_rmse = run.data.metrics["test_rmse"]

print(f"Test RMSE: {test_rmse}")

Test RMSE: 5.567408012462019
